In [10]:
import os
import requests
import PyPDF2
import pandas as pd

FILE_NAME_TPL = "{}_{}_Mobility_Report_en.pdf"
REPORT_START_DATE = "2020-02-16"
REPORT_DATE_END = "2020-04-9"
LABELS = [
    "Retail & recreation",
    "Grocery & pharmacy",
    "Parks",
    "Transit stations",
    "Workplaces",
    "Residential",
]

countries = pd.read_csv('data/countries_original.csv', na_filter=False)

# Download pdf

In [11]:
for code in countries['iso_alpha2']:
    path = "data/pdf/" + FILE_NAME_TPL.format(REPORT_DATE_END, code)
    print(path)
    if not os.path.isfile(path):
        #https://www.gstatic.com/covid19/mobility/2020-04-05_AF_Mobility_Report_en.pdf
        url = "https://www.gstatic.com/covid19/mobility/" + FILE_NAME_TPL.format(REPORT_DATE_END, code)
        print(url)
        r = requests.get(url, allow_redirects=True)
        print(r)
        if r.status_code == 404:
            continue

        print(code)
        with open(path, 'wb') as f:
            f.write(r.content)

data/pdf/2020-04-9_AF_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_AF_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_AL_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_AL_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_DZ_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_DZ_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_AD_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_AD_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_AO_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_AO_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_AG_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_AG_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_AZ_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_AZ_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_AR_Mobil

<Response [404]>
data/pdf/2020-04-9_GH_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_GH_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_GR_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_GR_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_GD_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_GD_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_GT_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_GT_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_GN_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_GN_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_GY_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_GY_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_HT_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_HT_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2

<Response [404]>
data/pdf/2020-04-9_PG_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_PG_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_PY_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_PY_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_PE_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_PE_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_PH_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_PH_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_PL_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_PL_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_PT_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_PT_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2020-04-9_TL_Mobility_Report_en.pdf
https://www.gstatic.com/covid19/mobility/2020-04-9_TL_Mobility_Report_en.pdf
<Response [404]>
data/pdf/2

# Parse pdf

In [4]:
def get_points(raw_data):
    points = []
    
    text = raw_data.decode("utf-8")
    prev_point = 60.0

    for line in text.split('\n'):
        p = line.split()
        if len(p) == 3:
            point = float(p[1])
            if point == 60.0 and prev_point == 60.0:
                continue
            else:
                prev_point = point

            points.append((60 - point)*1.33)

    return list(reversed(points))


def get_graphs_from_page(page):
    graphs = []

    for nobj in sorted([int(xobj.strip('/X')) for xobj in page['/Resources']['/XObject']]):
        raw_data = page['/Resources']['/XObject']['/X'+str(nobj)].getData()
        points = get_points(raw_data)
        graphs.append(points)

    return graphs

def get_graphs_from_pdf(pdf, columns):
    graphs = []
    graphs += get_graphs_from_page(pdf.getPage(0))
    graphs += get_graphs_from_page(pdf.getPage(1))
    
    if len(graphs) != len(columns):
        raise Exception('Invalid columns len')

    df = pd.DataFrame(graphs).T
    df.index = pd.date_range(start=REPORT_START_DATE, periods=len(df))
    df.columns = columns
    
    return df

In [5]:
data = []

for i, row in countries.iterrows():
    file_path = "data/pdf/" + FILE_NAME_TPL.format(REPORT_DATE_END, row["iso_alpha2"])
    if os.path.isfile(file_path):
        if row["iso_alpha2"]  == "LI":
            continue

        pdf = PyPDF2.PdfFileReader(open(file_path, "rb"))
        df = get_graphs_from_pdf(pdf, LABELS)

        for date, val in df.iterrows():
            data.append([date, row["iso_alpha3"]] + list(val.values))
            
df = pd.DataFrame(data, columns = ['date', 'country'] + LABELS)
df

,date,country,Retail & recreation,Grocery & pharmacy,Parks,Transit stations,Workplaces,Residential


In [6]:
def get_data_by_country(country):
    data_by_country = df[df["country"] == country]
    return pd.DataFrame(data_by_country[LABELS].values, index=data_by_country["date"], columns=LABELS)

get_data_by_country("IRL").plot(figsize=(15,7))

TypeError: no numeric data to plot

In [6]:
df.to_csv("data/mobility.csv", index=False)